# ゼミ第２回Python解析

色々とやってみる…？

In [ ]:
# !conda install ...
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_table("fcount.tsv")

In [ ]:
df.head(3)

### 整形

In [ ]:
samples=["LNCaP1", "LNCaP2", "LNCaP3", "MR49F1", "MR49F2", "MR49F3"]

del df["Chr"], df["Start"], df["End"], df["Strand"]

for i in range(6):
    df[samples[i]] = df["SRR82838"+str(i+87)+"_sorted_hisat.bam"]
    del df["SRR82838"+str(i+87)+"_sorted_hisat.bam"]

df

In [ ]:
# 発現量が全部0の行
allzeros = df[(df.iloc[:, 2:]==0).sum(axis=1)==(len(df.columns)-2)]
allzeros.head()

In [ ]:
df = df[(df.iloc[:, 2:]==0).sum(axis=1)<(len(df.columns)-2)]
df = df.reset_index(drop=True)
df

### 正規化


In [ ]:
rpm = df[samples].copy()

sums = rpm.sum()
sums

In [ ]:
rpm = 1e6 * rpm / sums # RPM化(全体の合計が1e6)
rpm.head()

In [ ]:
rpm.sum()

In [ ]:
def toRPM(df):
    d = df.copy()
    sums = d.sum()
    return 1e6 * d / sums

In [ ]:
def toRPKM(df, gene_lengths):
    d = toRPM(df)
    return (d.T / gene_lengths * 1e3).T # トリッキーな方法

In [ ]:
rpkm = toRPKM(df[samples], df["Length"])
rpkm.head()

In [ ]:
def toTPM(df, gene_lengths):
    counts = df.values
    length = gene_lengths.values
    counts_2 = counts / length.reshape(-1, 1) * 1e3
    return pd.DataFrame(counts_2 / counts_2.sum(axis=0) * 1e6, columns=df.columns)

In [ ]:
tpm = toTPM(df[samples], df["Length"])
tpm.head()

### プロット

In [ ]:
px.scatter_matrix(tpm)

In [ ]:
tpm["LNCaP_mean"] = np.mean(tpm[["LNCaP1","LNCaP2","LNCaP3"]], axis=1)
tpm["LNCaP_var"] = np.var(tpm[["LNCaP1","LNCaP2","LNCaP3"]], axis=1)
tpm["MR49F_mean"] = np.mean(tpm[["MR49F1","MR49F2","MR49F3"]], axis=1)
tpm["MR49F_var"] = np.var(tpm[["MR49F1","MR49F2","MR49F3"]], axis=1)
tpm["mean"] = np.mean(tpm, axis=1)
tpm["var"] = np.var(tpm, axis=1)
tpm.head()

In [ ]:
px.scatter(tpm, x="mean", y="var", log_x=True, log_y=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode="markers", x=tpm["LNCaP_mean"], y=tpm["LNCaP_var"], marker=dict(size=1), name="LNCaP"))
fig.add_trace(go.Scatter(mode="markers", x=tpm["MR49F_mean"], y=tpm["MR49F_var"], marker=dict(size=1), name="MR49F"))
fig.add_trace(go.Scatter(mode="markers", x=tpm["mean"], y=tpm["var"], marker=dict(size=1), name="All"))
fig.add_trace(go.Scatter(x=[1e-7, 1e7], y=[1e-7, 1e7], name="y=x"))
fig.update_xaxes(type="log", range=[-7, 7])
fig.update_yaxes(type="log", range=[-7, 7])
fig.show()

In [ ]:
rpm["LNCaP_mean"] = np.mean(rpm[["LNCaP1","LNCaP2","LNCaP3"]], axis=1)
rpm["LNCaP_var"] = np.var(rpm[["LNCaP1","LNCaP2","LNCaP3"]], axis=1)
rpm["MR49F_mean"] = np.mean(rpm[["MR49F1","MR49F2","MR49F3"]], axis=1)
rpm["MR49F_var"] = np.var(rpm[["MR49F1","MR49F2","MR49F3"]], axis=1)
rpm["mean"] = np.mean(rpm, axis=1)
rpm["var"] = np.var(rpm, axis=1)
rpm.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode="markers", x=rpm["LNCaP_mean"], y=rpm["LNCaP_var"], marker=dict(size=1), name="LNCaP"))
fig.add_trace(go.Scatter(mode="markers", x=rpm["MR49F_mean"], y=rpm["MR49F_var"], marker=dict(size=1), name="MR49F"))
fig.add_trace(go.Scatter(mode="markers", x=rpm["mean"], y=rpm["var"], marker=dict(size=1), name="All"))
fig.add_trace(go.Scatter(x=[1e-7, 1e7], y=[1e-7, 1e7], name="y=x"))
fig.update_xaxes(type="log", range=[-7, 7])
fig.update_yaxes(type="log", range=[-7, 7])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=tpm[tpm["LNCaP1"] <= 50]["LNCaP1"], name="LNCaP1", nbinsx=100, histnorm='probability'))
fig.add_trace(go.Histogram(x=tpm[tpm["LNCaP2"] <= 50]["LNCaP2"], name="LNCaP2", nbinsx=100, histnorm='probability'))
fig.add_trace(go.Histogram(x=tpm[tpm["LNCaP3"] <= 50]["LNCaP3"], name="LNCaP3", nbinsx=100, histnorm='probability'))
fig.add_trace(go.Histogram(x=tpm[tpm["MR49F1"] <= 50]["MR49F1"], name="MR49F1", nbinsx=100, histnorm='probability'))
fig.add_trace(go.Histogram(x=tpm[tpm["MR49F2"] <= 50]["MR49F2"], name="MR49F2", nbinsx=100, histnorm='probability'))
fig.add_trace(go.Histogram(x=tpm[tpm["MR49F3"] <= 50]["MR49F3"], name="MR49F3", nbinsx=100, histnorm='probability'))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

### サンプルの検定

In [ ]:
from scipy import stats
import statsmodels.stats.multitest as multi
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Spearman 相関係数
corr, p = stats.spearmanr(tpm.LNCaP1, tpm.LNCaP2)
print("corr:", corr, "p:", p)
corr, p = stats.spearmanr(tpm.LNCaP1, tpm.LNCaP3)
print("corr:", corr, "p:", p)
corr, p = stats.spearmanr(tpm.LNCaP2, tpm.LNCaP3)
print("corr:", corr, "p:", p)

corr, p = stats.spearmanr(tpm.MR49F1, tpm.MR49F2)
print("corr:", corr, "p:", p)
corr, p = stats.spearmanr(tpm.MR49F1, tpm.MR49F3)
print("corr:", corr, "p:", p)
corr, p = stats.spearmanr(tpm.MR49F2, tpm.MR49F3)
print("corr:", corr, "p:", p)

corr, p = stats.spearmanr(tpm.LNCaP1, tpm.MR49F1)
print("corr:", corr, "p:", p)
corr, p = stats.spearmanr(tpm.LNCaP2, tpm.MR49F2)
print("corr:", corr, "p:", p)

In [ ]:
# Mann-Whitney Utest

l1m1 = stats.mannwhitneyu(tpm.LNCaP1, tpm.MR49F1, alternative='two-sided')
l1m2 = stats.mannwhitneyu(tpm.LNCaP1, tpm.MR49F2, alternative='two-sided')
l1m3 = stats.mannwhitneyu(tpm.LNCaP1, tpm.MR49F3, alternative='two-sided')
l2m1 = stats.mannwhitneyu(tpm.LNCaP2, tpm.MR49F1, alternative='two-sided')
l2m2 = stats.mannwhitneyu(tpm.LNCaP2, tpm.MR49F2, alternative='two-sided')
l2m3 = stats.mannwhitneyu(tpm.LNCaP2, tpm.MR49F3, alternative='two-sided')
l3m1 = stats.mannwhitneyu(tpm.LNCaP3, tpm.MR49F1, alternative='two-sided')
l3m2 = stats.mannwhitneyu(tpm.LNCaP3, tpm.MR49F2, alternative='two-sided')
l3m3 = stats.mannwhitneyu(tpm.LNCaP3, tpm.MR49F3, alternative='two-sided')

l1l2 = stats.mannwhitneyu(tpm.LNCaP1, tpm.LNCaP2, alternative='two-sided')
l1l3 = stats.mannwhitneyu(tpm.LNCaP1, tpm.LNCaP3, alternative='two-sided')
l2l3 = stats.mannwhitneyu(tpm.LNCaP2, tpm.LNCaP3, alternative='two-sided')
m1m2 = stats.mannwhitneyu(tpm.MR49F1, tpm.MR49F2, alternative='two-sided')
m1m3 = stats.mannwhitneyu(tpm.MR49F1, tpm.MR49F3, alternative='two-sided')
m2m3 = stats.mannwhitneyu(tpm.MR49F2, tpm.MR49F3, alternative='two-sided')

print(l1m1.pvalue, l1m2.pvalue, l1m3.pvalue, l2m1.pvalue, l2m2.pvalue, l2m3.pvalue, l3m1.pvalue, l3m2.pvalue, l3m3.pvalue)
# 果たして replicate なのか
print(l1l2.pvalue, l1l3.pvalue, l2l3.pvalue, m1m2.pvalue, m1m3.pvalue, m2m3.pvalue)

発現変動の検定（雑）をやってみよう

In [ ]:
ngene = len(tpm.values)
pv = np.zeros(ngene)

for i in range(ngene):
    x = tpm.LNCaP1[i] + tpm.LNCaP2[i] + tpm.LNCaP3[i]
    y = tpm.MR49F1[i] + tpm.MR49F2[i] + tpm.MR49F3[i]
    
    if (tpm.iloc[i, :]==0).sum() > 1:
        pv[i] = 1
    else:
        chi = stats.chi2_contingency(np.array([[x,y],[3e6-x, 3e6-y]]))
        pv[i] = chi[1]

In [ ]:
bonf = multi.multipletests(pv, alpha=0.05, method='Bonferroni')
fdr = multi.multipletests(pv, alpha=0.05, method='fdr_bh')
print(bonf[1][:20])
print(fdr[1][:20])

In [ ]:
px.histogram(bonf[1], log_y=True)

In [ ]:
px.histogram(fdr[1],log_y=True)

In [ ]:
tests = df[["Geneid"]]
tests = tests.join(pd.DataFrame(bonf[1], columns=["Bonferroni"]))
tests = tests.join(pd.DataFrame(fdr[1], columns=["FDR"]))
tests

Homo_sapiens.GRCh38.104.gtf から "Gene_id" と "gene_name" の情報を抜き出したい！
でもgtfファイル自身は落としたくない…
→こういう時は shell で解決！

In [ ]:
#(シェル) tail -n +6 Homo_sapiens.GRCh38.104.gtf | awk -F "[ \t;]" '($3 == "gene" && $15 == "gene_name") { print $10"\t"$16 }' > id_names.tsv
# あとは scp なりで手元に落としてくる
# tsv ファイルの最初に "Geneid" と "name" を追加しておく(header)

In [ ]:
geneinfo = pd.read_table("id_names.tsv", header=None)
geneinfo.columns=["Geneid", "Name"]
geneinfo.head()

In [ ]:
tests_with_name = tests.merge(geneinfo)
# Name のないものはぶち消してる可能性がある…
tests_with_name

In [ ]:
tests_with_name[tests_with_name["Bonferroni"] < 0.05]

In [ ]:
tests_with_name[tests_with_name["FDR"] < 0.05]

In [ ]:
f = open("ChiSquared_Bonferroni.txt", 'w')
for name in tests_with_name[tests_with_name["Bonferroni"] < 0.05]["Name"]:
    f.write(name)
    f.write('\n')
f.close()

In [ ]:
tpm_with_id = df[["Geneid"]].join(tpm).merge(geneinfo)
chi_bonf_tpm = tpm_with_id[tpm_with_id["Geneid"].isin(tests[tests["Bonferroni"] < 0.05]["Geneid"])]

fig = go.Figure()
fig.add_trace(go.Scatter(mode="markers", x=np.log(tpm_with_id["mean"]), y=np.log(tpm_with_id["MR49F_mean"])-np.log(tpm_with_id["LNCaP_mean"]), name="all", marker=dict(size=2), hovertext=tpm_with_id["Name"]))
fig.add_trace(go.Scatter(mode="markers", x=np.log(chi_bonf_tpm["mean"]), y=np.log(chi_bonf_tpm["MR49F_mean"])-np.log(chi_bonf_tpm["LNCaP_mean"]), name="DEG", marker=dict(size=2), hovertext=chi_bonf_tpm["Name"]))
fig.show()

In [ ]:
f = open("ChiSquared_FDR.txt", 'w')
for name in tests_with_name[tests_with_name["FDR"] < 0.05]["Name"]:
    f.write(name)
    f.write('\n')
f.close()

In [ ]:
tpm_with_id = df[["Geneid"]].join(tpm).merge(geneinfo)
chi_fdr_tpm = tpm_with_id[tpm_with_id["Geneid"].isin(tests[tests["FDR"] < 0.05]["Geneid"])]

fig = go.Figure()
fig.add_trace(go.Scatter(mode="markers", x=np.log(tpm_with_id["mean"]), y=np.log(tpm_with_id["MR49F_mean"])-np.log(tpm_with_id["LNCaP_mean"]), name="all", marker=dict(size=2), hovertext=tpm_with_id["Name"]))
fig.add_trace(go.Scatter(mode="markers", x=np.log(chi_fdr_tpm["mean"]), y=np.log(chi_fdr_tpm["MR49F_mean"])-np.log(chi_fdr_tpm["LNCaP_mean"]), name="DEG", marker=dict(size=2), hovertext=chi_fdr_tpm["Name"]))
fig.show()

### よりガッツリ"Machine Learning"な方法で

Generalized linear models によって ポアソン分布 or 負の二項分布 を仮定した 最尤推定 とか AIC によるモデル選択など

In [ ]:
poisson = smf.glm("tpm.LNCaP1 ~ 1", data=tpm.LNCaP1, family=sm.families.Poisson())
res1 = poisson.fit()
res1.summary()

In [ ]:
NB = smf.glm("tpm.LNCaP1 ~ 1", data=tpm.LNCaP1, family=sm.families.NegativeBinomial())
res2 = NB.fit()
res2.summary()

In [ ]:
print(res1.aic, res2.aic)

edgeR および DESeq2 のような高度な解析ソフトは全部 R に集まっているので、Pythonではできません。